## tl;dr

Within training set we have `CVC - Abnormal` category. This category has a very broad possible cases what can go wrong with CVC placement in your body.

I manually inspected training data and split `CVC - Abnormal` category into two main ones:

- `CVC - Abnormal - Right Atrium` - most common abnormality, when CVC is inserted too deep into the heart
- `CVC - Abnormal - Other` - all other abnormalities, which include bad CVC line trajectory, CVC kinkings and other abnormalities.

The dataset is provided in https://www.kaggle.com/raddar/ranzcr-clip-abnormal-cvc-categories which maps to competition training data annotations by `index` column.

This information is potentially useful if you are going to train detectors.

## CVC positioning - how is it determined?

To show how normal/borderline/abnormal CVC position is decided, one can look at this illustration:

![](https://cdn.shopify.com/s/files/1/0667/1365/articles/What_is_a_picc_line_image-529932_600x.jpg?v=1587788405)


- Image above shows normal CVC tip placement.

- If CVC tip is little lower (but not too low) compared to reference image, the position is considered as borderline - neither normal nor abnormal.

- If CVC is much lower, the position is considered abnormal (malpositioned). This is also known as CVC tip placement in Right Atrium. However, this is only one type of possible abnormalities (although most common one).

- If the tip of CVC is above compared to reference image, this another kind of abnormality, caused due to many reasons of CVC insertion failures - line going in wrong venaes, CVC kinking, etc.


In [ ]:
import ast
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_colwidth = 1000

# my dataset
cvc_malpositions = pd.read_csv('../input/ranzcr-clip-abnormal-cvc-categories/cvc_malposition_categories.csv')

# competition data
train_ann = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train_annotations.csv')

I've spent some time to look through all `CVC - Abnormal` cases in order to split abnormalities into 2 distinct categories:

- `CVC - Abnormal - Right Atrium` - most common abnormality, when CVC is inserted too deep into the heart
- `CVC - Abnormal - Other` - all other abnormalities, which include bad CVC line trajectory, CVC kinkings and other abnormalities.

In [ ]:
# the table below can be joined with competition annotations by the `index` column
cvc_malpositions

In [ ]:
cvc_malpositions.groupby('label')['label'].count()

## Visualizations

### Normal cases

In [ ]:
def plot_xray(index):
    studyuid = train_ann.loc[index,'StudyInstanceUID']
    img = cv2.imread('../input/ranzcr-clip-catheter-line-classification/train/'+studyuid+'.jpg')
    cvc = ast.literal_eval(train_ann.loc[index,'data'])
    for c in cvc:
        img = cv2.circle(img, tuple(c), 5, (255,0,0), 5)
    plt.figure(figsize=(8,8))
    plt.imshow(img)

In [ ]:
plot_xray(16)

In [ ]:
plot_xray(7)

Both cases show CVC in correct position, with the tip of CVC being close to the heart.

### Borderline

In [ ]:
plot_xray(13)

In [ ]:
plot_xray(40)

The tip of CVC is within heart shadow, however radiologists did not consider the tip of CVC to be too deep into the heart. This is very subjective as radiologists could disagree if its borderline or not quite often.

### Abnormal - Right Atrium

Now we are going to inspect different types of abnormalities based on my manual annotations. Let's take a look at the most common one - the tip of CVC in heart's right atrium.

In [ ]:
plot_xray(25)

In [ ]:
plot_xray(42)

The tip of CVC is clearly way too deep into the heart. This is troublesome as this can have direct impact to the patient's health.

### Abnormal - Other

In [ ]:
plot_xray(34)

This a common abnormality, when the CVC trajectory goes into other venae - in this case to the neck. The CVC must be reinserted for this patient!

In [ ]:
plot_xray(43)

This is a typical case of CVC kinking - when the tip of CVC is kinked during insertion procedure.

## Different CVC - Abnormal categories - what's next?

I have shown that `CVC - Abnormal` category should not be treated as a homogenous feature of CVC - it clearly has many different forms of possible abnormalities.

This might be useful when validating classifiers. However, the biggest benefit might come if one builds detection/segmentation models.

After all, CVC classification task is the hardest problem of the competition - even small gains can lead to huge AUC differences on the LB.